In [22]:
import pandas as pd
from datetime import datetime

import os
import sys
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')
import src.utils.utils as util   # 기타 유틸리티 모듈


In [24]:
data_dir = '../outputs/intubation_extubation_modified_240122.csv'

intubation_extubation = pd.read_csv(data_dir, index_col=0)

In [3]:
hadm_id = 26919892

test_sub = intubation_extubation[intubation_extubation.hadm_id == hadm_id]
test_sub

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime
1476,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-28 05:59:00,224385.0,92.4,NaN,NaN,NaN,NaN,NaN,2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,NaN,0 days 00:00:00
1477,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-28 18:36:00,224385.0,92.4,30916265.0,2134-12-28 18:36:00,225477.0,92.4,Unplanned Extubation (non-patient initiated),2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,0 days 00:00:00,0 days 14:24:00
1478,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-29 09:00:00,224385.0,92.4,30916265.0,2134-12-29 14:57:00,227194.0,92.4,Planned Extubation,2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,0 days 05:57:00,0 days 00:05:00
1479,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-29 15:02:00,224385.0,92.4,NaN,NaN,NaN,NaN,NaN,2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,NaN,NaN
1480,11570331,26919892,30916265.0,2134-12-28 04:51:00,2135-01-03 17:15:00,224385.0,92.4,30916265.0,2135-01-04 08:54:00,225468.0,92.4,Unplanned Extubation (patient-initiated),2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,0 days 15:39:00,0 days 00:00:00


In [95]:
unique_intubations = list(test_sub['intubationtime'].dropna().unique())
unique_extubations = list(test_sub['extubationtime'].dropna().unique())

In [96]:
unique_intubations

['2134-12-28 05:59:00',
 '2134-12-28 18:36:00',
 '2134-12-29 09:00:00',
 '2134-12-29 15:02:00',
 '2135-01-03 17:15:00']

In [97]:
unique_extubations

['2134-12-28 18:36:00', '2134-12-29 14:57:00', '2135-01-04 08:54:00']

In [16]:
# 정렬 알고리즘 v3
def find_pairs(unique_intubations, unique_extubations):
    ''' 
    정렬 알고리즘 v3: 삽관 및 발관 시간 쌍 찾기
    이 함수는 고유한 삽관 시간 목록과 고유한 발관 시간 목록을 입력으로 받아
    삽관 시간과 발관 시간의 쌍을 찾습니다.

    매개변수:
    unique_intubations (list): 고유한 삽관 시간 목록
    unique_extubations (list): 고유한 발관 시간 목록

    반환값:
    list: 삽관 시간과 발관 시간의 쌍 목록
            (intubation_time, extubation_time) 형식의 튜플로 구성됨
            쌍을 찾지 못한 경우 (intubation_time, None) 또는 (None, extubation_time)으로 표시
    '''
    unique_intubations.sort()
    unique_extubations.sort()

    count = 0 # test code

    pairs = []   # 쌍을 저장할 빈 목록 생성
    int_index, ext_index = 0, 0   # 삽관 및 발관 시간 목록에서 현재 위치를 추적할 인덱스 변수

    # 두 목록 중 하나라도 끝에 도달할 때까지 반복
    while int_index < len(unique_intubations) or ext_index < len(unique_extubations):
        # 현재 삽관 및 발관 시간을 가져오고, 목록 끝에 도달한 경우 None 할당
        int_time = unique_intubations[int_index] if int_index < len(unique_intubations) else None
        ext_time = unique_extubations[ext_index] if ext_index < len(unique_extubations) else None

        # 케이스 1: 삽관 시간이 더 이상 없음
        if int_time is None:
            pairs.append((None, ext_time))
            ext_index += 1
        # 케이스 2: 발관 시간이 삽관 시간과 일치하거나 더 이상 발관 시간이 없음
        elif ext_time is None or ext_time >= int_time:
            # 발관 시간이 현재 삽관 시간에 유효한지 확인
            if ext_time is None or (int_index + 1 < len(unique_intubations) and ext_time >= unique_intubations[int_index + 1]):
                pairs.append((int_time, None))   # 쌍에 (삽관 시간, None) 추가
            else:
                pairs.append((int_time, ext_time))   # 쌍에 (삽관 시간, 발관 시간) 추가
                ext_index += 1   # 다음 발관 시간으로 이동
            int_index += 1   # 다음 삽관 시간으로 이동

            # test code
            if ext_time == int_time:
                print('found int_time ext_time match.')
                count += 1
        # 케이스 3: 발관 시간이 삽관 시간보다 이전임
        else:
            pairs.append((None, ext_time))   # 쌍에 (None, 발관 시간) 추가
            ext_index += 1   # 다음 발관 시간으로 이동

    return pairs


In [14]:
pairs = find_pairs(unique_intubations, unique_extubations)
pairs

[('2134-12-28 05:59:00', None),
 ('2134-12-28 18:36:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [17]:
group = intubation_extubation.groupby(['subject_id', 'hadm_id'])

def multi_row_formatting(group, subject_id, hadm_id):

    # (subject_id, hadm_id) 그룹 내에서 고유한 intubationtime, extubationtime 값 추출
    unique_intubations = list(group['intubationtime'].dropna().unique())
    unique_extubations = list(group['extubationtime'].dropna().unique())

    # Pairing (sorting algorithm)
    pairs = find_pairs(unique_intubations, unique_extubations)  # result: list of tuples

for _, g in group:
    multi_row_formatting(g, g.subject_id.unique(), g.hadm_id.unique())


found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.
found int_time ext_time match.


In [19]:
intubation_extubation[intubation_extubation.intubationtime == intubation_extubation.extubationtime].count()

subject_id          29
hadm_id             29
int_stayid          29
admittime           29
intubationtime      29
int_itemid          29
int_weight          29
ext_stayid          29
extubationtime      29
ext_itemid          29
ext_weight          29
extubationcause     29
dischtime           29
deathtime           11
marker               1
int_ext_timediff    29
reintubationtime    20
dtype: int64

In [22]:
from datetime import datetime

# v4
def find_pairs_with_priority(unique_intubations, unique_extubations):
    unique_intubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_intubations]
    unique_extubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_extubations]

    unique_intubations.sort()
    unique_extubations.sort()

    pairs = []  
    int_index, ext_index = 0, 0  

    while int_index < len(unique_intubations) or ext_index < len(unique_extubations):
        int_time = unique_intubations[int_index] if int_index < len(unique_intubations) else None
        ext_time = unique_extubations[ext_index] if ext_index < len(unique_extubations) else None

        if int_time is None:
            pairs.append((None, ext_time))
            ext_index += 1
        elif ext_time is None or ext_time > int_time:
            pairs.append((int_time, ext_time)) 
            int_index += 1
            ext_index += 1 if ext_time is not None else 0
        elif ext_time == int_time:
            # Check for a better match for int_time
            next_ext_index = ext_index + 1
            while next_ext_index < len(unique_extubations) and unique_extubations[next_ext_index] <= int_time:
                next_ext_index += 1
            if next_ext_index < len(unique_extubations):
                pairs.append((int_time, unique_extubations[next_ext_index]))
                int_index += 1
            else:
                pairs.append((int_time, ext_time))  # Use the identical timestamps pair as a last resort
                int_index += 1
                ext_index += 1
        else:
            pairs.append((None, ext_time))
            ext_index += 1

    # Convert datetime objects back to strings for output
    formatted_pairs = [(int_time.strftime('%Y-%m-%d %H:%M:%S') if int_time else None,
                        ext_time.strftime('%Y-%m-%d %H:%M:%S') if ext_time else None)
                       for int_time, ext_time in pairs]

    return formatted_pairs

# Running the modified algorithm
result_with_priority = find_pairs_with_priority(unique_intubations, unique_extubations)
result_with_priority


[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-28 18:36:00', '2134-12-29 14:57:00'),
 ('2134-12-29 09:00:00', '2135-01-04 08:54:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', None)]

In [23]:
def find_pairs_exclude_identical(unique_intubations, unique_extubations):
    unique_intubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_intubations]
    unique_extubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_extubations]

    unique_intubations.sort()
    unique_extubations.sort()

    pairs = []  
    int_index, ext_index = 0, 0  

    while int_index < len(unique_intubations) or ext_index < len(unique_extubations):
        int_time = unique_intubations[int_index] if int_index < len(unique_intubations) else None
        ext_time = unique_extubations[ext_index] if ext_index < len(unique_extubations) else None

        if int_time and ext_time and int_time == ext_time:
            # Identical timestamps found, choose which one to drop
            # Let's drop the intubation time for this example
            int_index += 1
            continue  # Skip to the next intubation time without creating a pair

        # Logic for pairing intubation and extubation times
        if int_time is None or (ext_time and ext_time < int_time):
            pairs.append((None, ext_time))
            ext_index += 1
        else:
            next_int_time = unique_intubations[int_index + 1] if int_index + 1 < len(unique_intubations) else None
            if not next_int_time or ext_time < next_int_time:
                pairs.append((int_time, ext_time))
                ext_index += 1 if ext_time else 0
            else:
                pairs.append((int_time, None))
            int_index += 1

    # Convert datetime objects back to strings for output
    formatted_pairs = [(int_time.strftime('%Y-%m-%d %H:%M:%S') if int_time else None,
                        ext_time.strftime('%Y-%m-%d %H:%M:%S') if ext_time else None)
                       for int_time, ext_time in pairs]

    return formatted_pairs

# Running the adjusted algorithm
adjusted_result_exclude_identical = find_pairs_exclude_identical(unique_intubations, unique_extubations)
adjusted_result_exclude_identical



[('2134-12-28 05:59:00', None),
 (None, '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [26]:
new_int = ['2134-12-28 05:59:00', '2134-12-29 09:00:00', '2134-12-29 15:02:00', '2135-01-03 17:15:00']
new_ext = ['2134-12-28 18:36:00', '2134-12-29 14:57:00', '2135-01-04 08:54:00']
additional_logic_test= find_pairs_exclude_identical(new_int, new_ext)
additional_logic_test

[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [27]:
# first iteration
unique_intubations = list(test_sub['intubationtime'].dropna().unique())
unique_extubations = list(test_sub['extubationtime'].dropna().unique())

adjusted_result_exclude_identical = find_pairs_exclude_identical(unique_intubations, unique_extubations)
adjusted_result_exclude_identical

[('2134-12-28 05:59:00', None),
 (None, '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

-----

새로운 로직 테스트:
-1. 페어링을 한번 한다. -> 중복값 제거됨
-2. 다시 한번 페어링을 해서 중복값이 제거된 행을 다시 합쳐준다 (필요시)


In [77]:
def find_pairs_exclude_identical_adjusted(unique_intubations, unique_extubations):
    unique_intubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_intubations]
    unique_extubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_extubations]

    unique_intubations.sort()
    unique_extubations.sort()

    pairs = []
    int_index, ext_index = 0, 0

    while int_index < len(unique_intubations) or ext_index < len(unique_extubations):
        int_time = unique_intubations[int_index] if int_index < len(unique_intubations) else None
        ext_time = unique_extubations[ext_index] if ext_index < len(unique_extubations) else None

        if int_time and ext_time and int_time == ext_time:
            # Identical timestamps found, choose which one to drop
            # Let's drop the intubation time for this example
            int_index += 1
            continue  # Skip to the next intubation time without creating a pair

        # Logic for pairing intubation and extubation times
        if int_time is None or (ext_time and ext_time < int_time):
            pairs.append((None, ext_time))
            ext_index += 1
        else:
            next_int_time = unique_intubations[int_index + 1] if int_index + 1 < len(unique_intubations) else None
            # Fixed comparison when next_int_time is None
            if next_int_time is None or (ext_time and ext_time < next_int_time):
                pairs.append((int_time, ext_time))
                ext_index += 1 if ext_time else 0
            else:
                pairs.append((int_time, None))
            int_index += 1

    # Convert datetime objects back to strings for output
    formatted_pairs = [(int_time.strftime('%Y-%m-%d %H:%M:%S') if int_time else None,
                        ext_time.strftime('%Y-%m-%d %H:%M:%S') if ext_time else None)
                       for int_time, ext_time in pairs]

    return formatted_pairs

# You can now use this adjusted function with your unique_intubations and unique_extubations lists
# adjusted_result = find_pairs_exclude_identical_adjusted(unique_intubations, unique_extubations)



In [78]:
# first iteration
unique_intubations = list(test_sub['intubationtime'].dropna().unique())
unique_extubations = list(test_sub['extubationtime'].dropna().unique())

# Running the adjusted algorithm (run 1)
adjusted_result_exclude_identical = find_pairs_exclude_identical_adjusted(unique_intubations, unique_extubations)

# prepare second iteration
int_time_list = []
ext_time_list = []
for pair in adjusted_result_exclude_identical:
    if pair[0] is not None:
        int_time_list.append(pair[0])
    if pair[1] is not None:
        ext_time_list.append(pair[1])

# second iteration
second_pair2 = find_pairs_exclude_identical_adjusted(int_time_list, ext_time_list)
second_pair2


[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [67]:
int_time_list

['2134-12-28 05:59:00',
 '2134-12-29 09:00:00',
 '2134-12-29 15:02:00',
 '2135-01-03 17:15:00']

In [68]:
ext_time_list

['2134-12-28 18:36:00', '2134-12-29 14:57:00', '2135-01-04 08:54:00']

In [69]:
# 2nd iteration

second_pair = find_pairs(int_time_list, ext_time_list)
second_pair

[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [70]:
# 2-2 iteration
second_pair2 = find_pairs_exclude_identical(int_time_list, ext_time_list)
second_pair2

[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [88]:
group = intubation_extubation.groupby(['subject_id', 'hadm_id'])
group_pairs = []

def multi_row_formatting(group, subject_id, hadm_id):

    # (subject_id, hadm_id) 그룹 내에서 고유한 intubationtime, extubationtime 값 추출
    unique_intubations = list(group['intubationtime'].dropna().unique())
    unique_extubations = list(group['extubationtime'].dropna().unique())

    # Pairing (sorting algorithm) run 1
    pairs = find_pairs_exclude_identical_adjusted(unique_intubations, unique_extubations)  # result: list of tuples

    # prepare second iteration
    int_time_list = []
    ext_time_list = []
    for pair in pairs:
        if pair[0] is not None:
            int_time_list.append(pair[0])
        if pair[1] is not None:
            ext_time_list.append(pair[1]) 

    # run code second time
    pairs = find_pairs_exclude_identical_adjusted(int_time_list, ext_time_list)   

    return pairs

for _, g in group:
    pairs = multi_row_formatting(g, g.subject_id.unique(), g.hadm_id.unique())
    group_pairs.append(pairs)


In [89]:
len(group_pairs)

7087

In [106]:
group_pairs[13]

[('2150-03-11 15:34:00', '2150-03-12 18:15:00'),
 (None, '2150-03-19 11:05:00'),
 ('2150-03-27 07:44:00', '2150-03-27 16:33:00'),
 ('2150-03-30 08:20:00', '2150-04-02 01:09:00')]

In [109]:
pair_counts = []
for pairs in group_pairs:
    count = len(pairs)
    pair_counts.append(count)

TypeError: 'list' object is not callable

In [86]:
test_sub

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime
1476,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-28 05:59:00,224385.0,92.4,NaN,NaN,NaN,NaN,NaN,2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,NaN,0 days 00:00:00
1477,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-28 18:36:00,224385.0,92.4,30916265.0,2134-12-28 18:36:00,225477.0,92.4,Unplanned Extubation (non-patient initiated),2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,0 days 00:00:00,0 days 14:24:00
1478,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-29 09:00:00,224385.0,92.4,30916265.0,2134-12-29 14:57:00,227194.0,92.4,Planned Extubation,2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,0 days 05:57:00,0 days 00:05:00
1479,11570331,26919892,30916265.0,2134-12-28 04:51:00,2134-12-29 15:02:00,224385.0,92.4,NaN,NaN,NaN,NaN,NaN,2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,NaN,NaN
1480,11570331,26919892,30916265.0,2134-12-28 04:51:00,2135-01-03 17:15:00,224385.0,92.4,30916265.0,2135-01-04 08:54:00,225468.0,92.4,Unplanned Extubation (patient-initiated),2135-01-06 10:38:00,2135-01-06 10:38:00,NaN,0 days 15:39:00,0 days 00:00:00


In [73]:
# first iteration
unique_intubations = list(test_sub['intubationtime'].dropna().unique())
unique_extubations = list(test_sub['extubationtime'].dropna().unique())

# Running the adjusted algorithm (run 1)
adjusted_result_exclude_identical = find_pairs_exclude_identical(unique_intubations, unique_extubations)

# prepare second iteration
int_time_list = []
ext_time_list = []
for pair in adjusted_result_exclude_identical:
    if pair[0] is not None:
        int_time_list.append(pair[0])
    if pair[1] is not None:
        ext_time_list.append(pair[1])

# second iteration
second_pair2 = find_pairs_exclude_identical(int_time_list, ext_time_list)
second_pair2

[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [47]:
unique_extubations

['2134-12-28 18:36:00', '2134-12-29 14:57:00', '2135-01-04 08:54:00']

-----

In [110]:
group

[('2131-01-11 04:30:00', '2131-01-12 17:40:00')]

In [111]:
group = intubation_extubation.groupby(['subject_id', 'hadm_id'])
for _, g in group:
    if (g.intubationtime == g.extubationtime).sum() > 1:
        print('yes')
    


In [112]:
unique_intubations

['2134-12-28 05:59:00',
 '2134-12-28 18:36:00',
 '2134-12-29 09:00:00',
 '2134-12-29 15:02:00',
 '2135-01-03 17:15:00']

In [113]:
unique_extubations

['2134-12-28 18:36:00', '2134-12-29 14:57:00', '2135-01-04 08:54:00']

----

In [211]:
# v4 (intubationtime == extubationtime 케이스 고려)
def primary_pairing(unique_intubations, unique_extubations):
    """
    고유한 intubationtime, extubationtime 리스트를 받아 1차적으로 페어링 수행. intubationtime, extubationtime 중복값 발생 시 다음 함수로 넘김
    """

    # 문자열을 datetime 객체로 변환
    unique_intubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_intubations]
    unique_extubations = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in unique_extubations]

    # 시간 정렬
    unique_intubations.sort()
    unique_extubations.sort()

    pairs = []   # 페어링 저장할 리스트
    skipped_int_times = []   # 중복되는 intubation 이벤트 저장
    skipped_ext_times = []   # 중복되는 extubation 이벤트 저장
    int_index, ext_index = 0, 0

    # 페어링 로직
    # 두 목록 중 하나라도 끝에 도달할 때까지 반복
    while int_index < len(unique_intubations) or ext_index < len(unique_extubations):
        # 현재 삽관 및 발관 시간을 가져오고, 목록 끝에 도달한 경우 (적절한 값이 없는 경우) None 할당
        int_time = unique_intubations[int_index] if int_index < len(unique_intubations) else None
        ext_time = unique_extubations[ext_index] if ext_index < len(unique_extubations) else None

        if int_time == ext_time:
            # intubationtime과 extubationtime이 동일할 경우, "skipped" 리스트에 저장 후 일단 넘어감
            skipped_int_times.append(int_time)
            skipped_ext_times.append(ext_time)
            int_index += 1
            ext_index += 1
            continue

        # 적절한 intubationtime이 없는 경우, None 할당 (1.쓸 수 있는 int_time이 없거나, 2. int_time과 ext_time 시간 순서가 맞지 않을 경우)
        if int_time is None or (ext_time and ext_time < int_time):
            pairs.append((None, ext_time))
            ext_index += 1
        
        # 다음 intubationtime 과 비교
        else:
            next_int_time = unique_intubations[int_index + 1] if int_index + 1 < len(unique_intubations) else None

            # 1. 마지막 이벤트거나, 2. ext_time 이 다음 int_time 보다 앞에 오는 경우 -> 페어링 성공
            if next_int_time is None or (ext_time and ext_time < next_int_time):
                pairs.append((int_time, ext_time))
                ext_index += 1 if ext_time else 0
            
            # 위의 if 문의 조건을 충족하지 못했을 때, none 할당
            else:
                pairs.append((int_time, None))
            int_index += 1

    # datetime 객체를 문자열로 다시 변환
    formatted_pairs = [(int_time.strftime('%Y-%m-%d %H:%M:%S') if int_time else None,
                        ext_time.strftime('%Y-%m-%d %H:%M:%S') if ext_time else None)
                       for int_time, ext_time in pairs]

    formatted_skipped_int = [int_time.strftime('%Y-%m-%d %H:%M:%S') if int_time else None 
                             for int_time in skipped_int_times]
    formatted_skipped_ext = [ext_time.strftime('%Y-%m-%d %H:%M:%S') if ext_time else None 
                            for ext_time in skipped_ext_times]

    return formatted_pairs, formatted_skipped_int, formatted_skipped_ext


def integrate_skipped_times_into_pairs(pairs, skipped_int_times, skipped_ext_times):
    """ 
    이전 함수에서 intubationtime과 extubationtime이 동일한 경우 페어링 하지 않고, 'skipped' 리스트에 저장함.
    본 함수는 skipped 이벤트들을 페어링된 데이터와 비교하여 적절한 위치가 있을 경우 삽입해줄 것임. 
    """

    # datetime 객체로 변환 (문자열인 경우)
    skipped_int_times = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') if time else None for time in skipped_int_times]
    skipped_ext_times = [datetime.strptime(time, '%Y-%m-%d %H:%M:%S') if time else None for time in skipped_ext_times]

    # datetime 객체로 변환 (문자열인 경우)
    formatted_pairs = []
    for int_time, ext_time in pairs:
        formatted_int_time = datetime.strptime(int_time, '%Y-%m-%d %H:%M:%S') if int_time else None
        formatted_ext_time = datetime.strptime(ext_time, '%Y-%m-%d %H:%M:%S') if ext_time else None
        formatted_pairs.append((formatted_int_time, formatted_ext_time))

    # skipped intubationtime 케이스를 잘 페어링된 케이스들과 비교
    for skipped_int in skipped_int_times:
        if skipped_int is not None:
            for i, (int_time, ext_time) in enumerate(formatted_pairs):

                # int_time의 위치가 비어 있고, skipped intubationtime이 extubationtime 보다 앞에 올때 삽입
                if int_time is None and (ext_time is None or skipped_int < ext_time):
                    formatted_pairs[i] = (skipped_int, ext_time)
                    break

    # skipped extubationtime 케이스를 잘 페어링된 케이스들과 비교
    for skipped_ext in skipped_ext_times:
        if skipped_ext is not None:
            for i, (int_time, ext_time) in enumerate(formatted_pairs):

                # ext_time의 위치가 비어 있고, skipped extubationtime이 intubationtime 보다 앞에 올때 삽입
                if ext_time is None and (int_time is None or skipped_ext > int_time):
                    formatted_pairs[i] = (int_time, skipped_ext)
                    break

    # intubationtime과 extubationtime이 동일한데, 다른 짝이 없는 경우
    if not formatted_pairs and skipped_int_times and skipped_ext_times:
        # 동일한 값이지만 하나의 페어로 묶어줌 (추후 검증 코드에서 제거)
        for int_time, ext_time in zip(skipped_int_times, skipped_ext_times):
            formatted_pairs.append((int_time, ext_time))

    # datetime 객체를 문자열로 다시 변환
    formatted_pairs = [(int_time.strftime('%Y-%m-%d %H:%M:%S') if isinstance(int_time, datetime) else int_time,
                        ext_time.strftime('%Y-%m-%d %H:%M:%S') if isinstance(ext_time, datetime) else ext_time)
                       for int_time, ext_time in formatted_pairs]

    return formatted_pairs



def main_pairing(unique_intubations, unique_extubations):
    """
    앞의 두 함수를 사용해 페어링을 수행해주는 메인코드.
    """

    # 1차 페어링: 시간이 겹치는 이벤트를 따로 떼어놓고 페어링 수행
    primary_pairs, formatted_skipped_int, formatted_skipped_ext = primary_pairing(unique_intubations, unique_extubations)

    # 2차 페어링: 페어링된 데이터에 시간이 겹치는 이벤트들을 삽입 (적절한 위치가 있을 경우)
    all_pairs = integrate_skipped_times_into_pairs(primary_pairs, formatted_skipped_int, formatted_skipped_ext)

    return all_pairs


In [212]:
# test code
error_sub_ids = list(intubation_extubation[intubation_extubation.intubationtime == intubation_extubation.extubationtime].hadm_id)

testdf = intubation_extubation[intubation_extubation.hadm_id.isin(error_sub_ids)]

group = testdf.groupby(['subject_id', 'hadm_id'])
group_pairs = []

for _, g in group:
    # (subject_id, hadm_id) 그룹 내에서 고유한 intubationtime, extubationtime 값 추출
    unique_intubations = list(g['intubationtime'].dropna().unique())
    unique_extubations = list(g['extubationtime'].dropna().unique())

    pairs = main_pairing(unique_intubations, unique_extubations)
    group_pairs.append(pairs)

pair_lens = []
for pairs in group_pairs:
    pair_lens.append(len(pairs))

pd.Series(pair_lens).describe()



count    29.000000
mean      1.689655
std       1.072495
min       1.000000
25%       1.000000
50%       1.000000
75%       2.000000
max       5.000000
dtype: float64

In [204]:
testdf.hadm_id.unique()

array([24995523, 29747297, 28133020, 26919892, 25740306, 24994500,
       24529183, 21071536, 23772093, 27831438, 21093920, 24067663,
       27550181, 24657887, 23724954, 29601819, 27841468, 28324288,
       29715756, 26412554, 28452358, 24923181, 25060449, 22999142,
       20069079, 26865488, 26432998, 25847021, 24829358])

In [205]:
group_pairs[3]

[('2134-12-28 05:59:00', '2134-12-28 18:36:00'),
 ('2134-12-29 09:00:00', '2134-12-29 14:57:00'),
 ('2134-12-29 15:02:00', None),
 ('2135-01-03 17:15:00', '2135-01-04 08:54:00')]

In [206]:
group_pairs[26]

[('2188-01-09 10:45:00', '2188-01-10 07:45:00'),
 ('2188-01-10 07:45:00', '2188-01-12 11:48:00'),
 (None, '2188-01-15 17:57:00')]

In [207]:
testdf[testdf.hadm_id == 21071536]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime
2212,12393609,21071536,37742636.0,2124-07-20 22:04:00,2124-07-22 12:15:00,224385.0,120.6,NaN,NaN,NaN,NaN,NaN,2124-08-25 13:47:00,NaN,NaN,NaN,0 days 00:00:00
2213,12393609,21071536,37742636.0,2124-07-20 22:04:00,2124-08-02 12:15:00,224385.0,120.6,37742636.0,2124-08-02 12:15:00,227194.0,120.6,Planned Extubation,2124-08-25 13:47:00,NaN,NaN,0 days 00:00:00,2 days 06:01:00
2214,12393609,21071536,37742636.0,2124-07-20 22:04:00,2124-08-04 18:16:00,224385.0,120.6,NaN,NaN,NaN,NaN,NaN,2124-08-25 13:47:00,NaN,NaN,NaN,NaN
2215,12393609,21071536,NaN,2124-07-20 22:04:00,NaN,NaN,NaN,37742636.0,2124-08-04 15:00:00,227194.0,120.6,Planned Extubation,2124-08-25 13:47:00,NaN,NaN,NaN,0 days 00:00:00


In [ ]:
intubation = ['2124-07-22 12:15:00', '2124-08-02 12:15:00', '2124-08-04 18:16:00']
extubation = ['2124-08-02 12:15:00', '2124-08-04 15:00:00']

In [157]:
testdf

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime
255,10295020,24995523,39726152.0,2147-06-08 21:04:00,2147-06-08 23:05:00,224385.0,49.2,39726152.0,2147-06-10 17:06:00,225468.0,49.2,Unplanned Extubation (patient-initiated),2147-06-16 16:56:00,NaN,NaN,1 days 18:01:00,0 days 00:00:00
256,10295020,24995523,39726152.0,2147-06-08 21:04:00,2147-06-10 17:41:00,224385.0,49.2,39726152.0,2147-06-10 17:41:00,225468.0,49.2,Unplanned Extubation (patient-initiated),2147-06-16 16:56:00,NaN,NaN,0 days 00:00:00,NaN
257,10295020,24995523,NaN,2147-06-08 21:04:00,NaN,NaN,NaN,39726152.0,2147-06-14 10:45:00,227194.0,49.2,Planned Extubation,2147-06-16 16:56:00,NaN,NaN,NaN,0 days 00:00:00
1232,11300581,29747297,35776863.0,2120-03-09 18:07:00,2120-03-22 03:30:00,224385.0,103.0,35776863.0,2120-03-27 15:13:00,227194.0,103.0,Planned Extubation,2120-06-01 15:30:00,NaN,NaN,5 days 11:43:00,0 days 00:00:00
1233,11300581,29747297,35776863.0,2120-03-09 18:07:00,2120-03-30 13:40:00,224385.0,103.0,35776863.0,2120-03-30 13:40:00,227194.0,103.0,Planned Extubation,2120-06-01 15:30:00,NaN,NaN,0 days 00:00:00,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8729,19330004,26432998,NaN,2187-12-30 16:33:00,NaN,NaN,NaN,33297641.0,2188-01-15 17:57:00,227194.0,69.0,Planned Extubation,2188-01-21 13:40:00,NaN,NaN,NaN,0 days 00:00:00
8995,19576807,25847021,39472944.0,2133-10-15 21:31:00,2133-10-26 21:13:00,224385.0,110.8,39472944.0,2133-10-26 21:13:00,225477.0,110.8,Unplanned Extubation (non-patient initiated),2133-10-29 03:52:00,2133-10-29 03:52:00,NaN,0 days 00:00:00,0 days 00:00:00
8996,19576807,25847021,39472944.0,2133-10-15 21:31:00,2133-10-28 08:55:00,224385.0,110.8,39472944.0,2133-10-28 18:30:00,227194.0,110.8,Planned Extubation,2133-10-29 03:52:00,2133-10-29 03:52:00,NaN,0 days 09:35:00,0 days 00:00:00
9354,19929426,24829358,32905110.0,2189-10-30 16:33:00,2189-11-02 09:35:00,224385.0,80.0,32905110.0,2189-11-02 09:35:00,227194.0,80.0,Planned Extubation,2189-11-15 13:44:00,NaN,NaN,0 days 00:00:00,0 days 00:00:00


In [123]:
pair_lens = []
for pairs in group_pairs:
    pair_lens.append(len(pairs))

pd.Series(pair_lens).describe()

count    7087.000000
mean        1.379850
std         0.657084
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         6.000000
dtype: float64

In [210]:
pd.Series(pair_lens).value_counts()

1    13
2     5
0     5
3     4
4     1
5     1
Name: count, dtype: int64

In [124]:
import random

# Flatten the group_pairs list
flattened_pairs = [pair for sublist in group_pairs for pair in sublist]

# Randomly sample 100 pairs if there are enough pairs, otherwise sample all available pairs
sample_size = min(100, len(flattened_pairs))
sampled_pairs = random.sample(flattened_pairs, sample_size)

# sampled_pairs now contains 100 (or less, if there weren't enough) randomly chosen pairs


In [128]:
for i, p in enumerate(sampled_pairs):
    print(f'id: {i}')
    print(p)

id: 0
('2168-05-15 11:18:00', '2168-05-18 12:07:00')
id: 1
('2140-09-09 22:35:00', '2140-09-10 09:30:00')
id: 2
('2153-08-02 13:48:00', '2153-08-12 13:47:00')
id: 3
('2113-06-11 21:33:00', None)
id: 4
('2129-03-11 03:07:00', None)
id: 5
(None, '2133-01-24 16:30:00')
id: 6
('2175-08-02 03:44:00', None)
id: 7
('2129-02-18 19:24:00', '2129-02-20 19:30:00')
id: 8
('2159-11-02 09:29:00', '2159-11-03 15:49:00')
id: 9
('2147-11-11 08:35:00', '2147-11-12 12:02:00')
id: 10
('2189-08-23 11:18:00', '2189-08-30 13:06:00')
id: 11
('2118-08-29 07:10:00', '2118-08-30 10:17:00')
id: 12
('2114-09-06 19:30:00', None)
id: 13
(None, '2180-03-20 17:04:00')
id: 14
('2162-06-23 17:10:00', '2162-06-27 14:56:00')
id: 15
('2135-02-24 10:00:00', None)
id: 16
('2110-05-21 18:45:00', None)
id: 17
('2178-01-18 08:40:00', None)
id: 18
('2194-10-05 21:00:00', None)
id: 19
('2136-01-24 18:30:00', '2136-02-10 12:15:00')
id: 20
('2154-07-13 11:45:00', '2154-07-18 20:00:00')
id: 21
('2164-08-25 14:11:00', None)
id: 22
('

In [131]:
group_pairs[600]

[('2167-03-01 10:55:00', '2167-03-01 18:07:00')]

In [132]:
intubation_extubation

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,int_ext_timediff,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,1 days 13:10:00,0 days 00:00:00
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,3 days 18:19:00,0 days 00:00:00
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,NaN,0 days 00:00:00
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,2 days 17:30:00,0 days 00:00:00
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,NaN,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9414,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,5 days 20:16:00,0 days 00:00:00
9415,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,2 days 23:09:00,NaN
9417,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,2 days 22:01:00,0 days 00:00:00
9418,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,0 days 19:15:00,0 days 00:00:00


# 24. 1. 27

In [32]:
 # 데이터 reimport 할 경우 시간변수 변환

reintubation_df = pd.read_csv('../outputs/reintubation_df.csv', index_col=0)

time_cols = ['intubationtime', 'extubationtime', 'admittime', 'dischtime', 'deathtime']
reintubation_df = util.to_datetime(reintubation_df, time_cols)

Converted intubationtime to datetime.
Converted extubationtime to datetime.
Converted admittime to datetime.
Converted dischtime to datetime.
Converted deathtime to datetime.


In [33]:
reintubation_df

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaT,NaN,NaN,NaN,2144-04-13 17:31:00,NaT,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaT,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaT,NaN,NaN,NaN,2144-02-06 11:45:00,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaT,NaN,8416.0,2126-05-05 18:20:00,4430.0
9004,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaT,NaN,4269.0,NaN,NaN
9005,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaT,NaN,4201.0,NaN,NaN
9006,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaT,NaN,1155.0,NaN,NaN


In [34]:
reintubation_df[reintubation_df.intubationtime > reintubation_df.extubationtime]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime


In [12]:
mismatch_hadms = reintubation_df[reintubation_df.intubationtime > reintubation_df.extubationtime].hadm_id.unique()

In [29]:
reintubation_df[reintubation_df.intubationtime > reintubation_df.extubationtime]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
8,10011365,26712576,37153661.0,2166-02-11 14:09:00,2166-02-14 17:59:00,224385.0,46.3,37153661.0,2166-02-14 09:48:00,227194.0,46.3,Planned Extubation,2166-02-20 00:20:00,2166-02-20 00:20:00,NaN,-491.0,NaN,NaN
12,10018081,21027282,37293400.0,2133-12-18 16:58:00,2133-12-21 05:06:00,224385.0,91.0,37293400.0,2133-12-21 05:05:00,227194.0,91.0,Planned Extubation,2134-01-12 11:00:00,NaT,NaN,-1.0,NaN,NaN
18,10020944,29974575,30757476.0,2131-02-27 15:34:00,2131-03-03 20:11:00,224385.0,107.0,30757476.0,2131-03-03 19:54:00,225468.0,107.0,Unplanned Extubation (patient-initiated),2131-03-13 17:01:00,NaT,NaN,-17.0,NaN,NaN
40,10049041,25320808,33222233.0,2163-12-19 14:48:00,2163-12-20 18:35:00,224385.0,70.5,33222233.0,2163-12-20 15:45:00,227194.0,70.5,Planned Extubation,2164-01-08 17:32:00,NaT,NaN,-170.0,NaN,NaN
94,10123036,29540743,30112389.0,2168-06-06 02:32:00,2168-06-08 03:36:00,224385.0,95.4,35987620.0,2168-06-06 11:27:00,227194.0,95.4,Planned Extubation,2168-06-19 18:00:00,NaT,NaN,-2409.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9339,19940147,29892428,32144801.0,2127-11-11 12:28:00,2127-11-16 18:05:00,224385.0,64.0,32144801.0,2127-11-16 17:30:00,225468.0,64.0,Unplanned Extubation (patient-initiated),2127-11-25 16:00:00,NaT,NaN,-35.0,NaN,NaN
9356,19964153,20368705,34002206.0,2135-12-28 02:40:00,2136-01-10 21:49:00,224385.0,94.0,34002206.0,2135-12-31 13:49:00,227194.0,94.0,Planned Extubation,2136-01-11 06:00:00,2136-01-11 06:00:00,NaN,-14880.0,NaN,NaN
9381,19986715,28333468,30045509.0,2150-11-29 02:47:00,2150-12-16 12:30:00,224385.0,39.5,39855146.0,2150-12-08 09:30:00,227194.0,39.5,Planned Extubation,2151-01-14 14:20:00,NaT,NaN,-11700.0,NaN,NaN
9382,19989783,26984195,32761676.0,2130-07-12 08:34:00,2130-07-19 16:50:00,224385.0,101.0,32761676.0,2130-07-19 12:40:00,227194.0,101.0,Planned Extubation,2130-08-01 15:50:00,NaT,NaN,-250.0,NaN,NaN


In [30]:
len(reintubation_df[reintubation_df.hadm_id.isin(mismatch_hadms)].hadm_id.unique())

388

In [31]:
reintubation_df

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,mvtime,reintubation_eventtime,reintubationtime
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,2230.0,NaN,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,5419.0,NaN,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaT,NaN,NaN,NaN,2144-04-13 17:31:00,NaT,NaN,NaN,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaT,NaN,3930.0,2144-01-30 13:00:00,30.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaT,NaN,NaN,NaN,2144-02-06 11:45:00,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-04-26 20:14:00,224385.0,59.0,35616526.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaT,NaN,8416.0,2126-05-05 18:20:00,4430.0
9392,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaT,NaN,4269.0,NaN,NaN
9393,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaT,NaN,4201.0,NaN,NaN
9394,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaT,NaN,1155.0,NaN,NaN
